In [1]:
# %matplotlib inline
import gym, math, glob
import numpy as np

import cv2

from timeit import default_timer as timer
from datetime import timedelta

from collections import deque
import random

import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from IPython import display
from matplotlib import pyplot as plt
get_ipython().run_line_magic('matplotlib', 'inline')

# from utils.wrappers import *
# from agents.DQN import Model as DQN_Agent
# from utils.ReplayMemory import ExperienceReplayMemory

#from utils.Replay.ipynb import ReplayBuffer
# from utils.hyperparameters import Config
# from utils.plot import plot_all_data
# import Game.tetris_fun as game
from Game.tetris_env import *

ModuleNotFoundError: No module named 'utils.plot'

In [ ]:
# Solution for error: no available video device
import os
os.environ["SDL_VIDEODRIVER"] = "dummy"

In [ ]:
class ReplayBuffer:
    
    def __init__(self, size, screen_shape=(84, 84), frame_stack=True):
        self.size = size
        self.screen_shape = screen_shape
        self.num_in_buffer = 0
        self.memory = deque(maxlen=self.size)
#         self.actions = deque(maxlen=self.size)
#         self.rewards = deque(maxlen=self.size)
# #          self.next_screens = deque(maxlin=self.size)
#         self.terminal = deque(maxlen=self.size)
        
    def push(self, screen, action, reward, next_state, terminal):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        terminal = 0.0 if not terminal else 1.0
        screen = torch.from_numpy(screen).to(dtype=torch.float32, device=device)
#         print("action type", (action, type(action[0])))
        next_state = torch.from_numpy(next_state).to(dtype=torch.float32, device=device)
        action = action.to(device)
        reward = torch.tensor(reward).to(device)
        terminal = torch.tensor(terminal).to(device)
        self.memory.append((screen, action, reward, next_state, terminal))
#         self.actions.append(action)
#         self.rewards.append(reward)
#         self.terminal.append(terminal)
#         self.next_screens.append(next_screens)
        
        self.num_in_buffer = len(self.memory)
#         print("buffer num ", self.num_in_buffer)
#         print("action num ", len(self.actions))
#         print("reward num ", len(self.rewards))
#         print("ter num ", len(self.terminal))
#         print(self.actions)
        
    def can_sample(self, batch_size):
        """Returns true if `batch_size` different transitions can be sampled from the buffer."""
        return batch_size + 1 <= self.num_in_buffer
    
    def _encode_sample(self, idxes):
        # Return batch data for screens, actions, rewards, next screens and terminal info
        # one screen state corresponding to one action by default, needing to consider grouped screens and actions
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        samples = [self.memory[i] for i in idxes]
        obs_batch, act_batch, rew_batch, next_obs_batch, done_mask = zip(*samples)  # unzip the samples and return tuples
        print("sample zipped", obs_batch[0])
        print("act", act_batch[0])
        print("rew", rew_batch[0])
        print("next", next_obs_batch[0])
        print("done",done_mask)
        obs_batch, act_batch, rew_batch, next_obs_batch, done_mask = map(torch.stack, zip(*samples))  # convert tuple into tensor form
        print("obs_batch size", obs_batch.shape)
        print("act batch size", act_batch.shape)
        print("next b size", next_obs_batch.shape)
        print(done_mask)
        
#         obs_batch      = torch.from_numpy(np.concatenate([self.screens[idx] for idx in idxes], 0)).to(device)
#         act_batch      = torch.tensor([[self.actions[idx]] for idx in idxes]).type(torch.long).to(device)
#         rew_batch      = torch.tensor([self.rewards[idx] for idx in idxes]).to(device)
#         next_obs_batch = torch.from_numpy(np.concatenate([self.screens[idx + 1] for idx in idxes], 0)).to(device)
#         done_mask      = np.array([1.0 if self.terminal[idx] else 0.0 for idx in idxes], dtype=np.float32)
        
        return obs_batch, act_batch, rew_batch, next_obs_batch, done_mask
        
    
    def sample(self, batch_size):
#         assert self.can_sample(batch_size)
        inds = random.sample(range(self.num_in_buffer), batch_size)  # avoid sampling the last item and leading to index out of range for next state
#         print("ind ", inds)
        
        return self._encode_sample(inds)
        

In [ ]:
# The DQN model
class DQN(nn.Module):
    def __init__(self, num_actions, in_channels=4, screen_shape=(84, 84)):
        super(DQN, self).__init__()
        in_channels = in_channels
        num_actions = num_actions
        screen_shape = screen_shape
        h = screen_shape[0]
        w = screen_shape[1]
        
        
        # could add batchnorm2d layers after each covnet if data volume is too large
        # see: https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html
        
        self.conv1 = nn.Conv2d(in_channels, 32, kernel_size=8, stride=4)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1)
        
        convw = self.conv2d_size_out(self.conv2d_size_out(self.conv2d_size_out(w, 8, 4), 4, 2), 3, 1)
        convh = self.conv2d_size_out(self.conv2d_size_out(self.conv2d_size_out(h, 8, 4), 4, 2), 3, 1)
        
        self.fc1 = nn.Linear(convw * convh * 64, 512)
        self.fc2 = nn.Linear(512, num_actions)
        
        
    def forward(self, x):
        x = self.conv1(x)
#         print("forward", x.size())
        x = F.relu(x)
#         x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
#         print("forward", x.size())
        x = F.relu(self.conv3(x))
#         print("fc size in", x.size())
        # reshape the tensor to one dimension for fc layers
        x = x.view(x.size(0), -1)
#         print("fc size in", x.size())

        x = F.relu(self.fc1(x))

        return self.fc2(x)
    
    def conv2d_size_out(self, size, kernel_size, stride, padding_size=0):
        return (size + 2 * padding_size - (kernel_size - 1) - 1) // stride  + 1
    

# m = DQN(6, 1, (84, 84))
# input1 = torch.randn(1, 1, 84, 84)
# print(input1.dtype)
# output = m(input1)

# x_t = np.array([1, 2, 255, 3, 8, 255], dtype=np.double)
# x_t = np.array([0.0 if x//255 < 1 else 255.0 for x in x_t])
# print(x_t)

    

In [ ]:
# Hyperparameters and utilities

BATCH_SIZE = 128
GAMMA = 0.999
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 0.001
TARGET_UPDATE = 10
lr = 0.001
memory_size = 100000
num_episodes = 1000


def get_action(state, policy_net):
    # Return a number indicating the pos of 1 in the array for a action
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    steps_done = 0
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            # t.max(1) will return largest column value of each row.
            # second column on max result is index of where max element was
            # found, so we pick action with the larger expected reward.
            
            state = torch.from_numpy(state)
            if state.ndim < 4:
                state = state.unsqueeze(0)
            action = policy_net(state.to(device, dtype=torch.float32))
#             print("act type", type(action))
#             print(action)
#             action = action.max(1)[1][0]
            action = action.max(1)[1]
#             print("act num size", action.size())
            return action
    else:
        action = torch.tensor([random.randint(0, 5)])
#         print("rand int size ", action.size())
        return action
    
# def get_act_array(act_num):
#     act_num = int(act_num.item())
# #     print("act_num", act_num)
#     action = np.zeros(6, dtype=int)
#     action[act_num] = 1
#     return action

def get_next_qs(target_net, next_obs_batch, done_mask, BATCH_SIZE):
    """
    Return the Q-value of the next state.
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    terminal_ind = torch.nonzero(done_mask)
    values = target_net(next_obs_batch.to(device)).max(dim=1)[0].detach() 
#     print("values", values.shape)
#     print(terminal_ind)
#     print("values val", values)
    values[terminal_ind] = 0.0
#     print("values with ter", values)
    
    
#     not_terminal = np.where(done_mask==0.0)  # find elements fulfill the condition(==0.0), and return a tuple with an array of their positions
#     not_terminal_states = next_obs_batch[not_terminal]
#     values = torch.zeros(BATCH_SIZE).to(device)
#     values[not_terminal] = target_net(not_terminal_states.to(device)).max(dim=1)[0].detach()
    return values
            
# No need due to Wrapper added
def preprocess(image, shape=(84,84)):
    x_t = image
    x_t = cv2.cvtColor(cv2.resize(x_t, shape), cv2.COLOR_BGR2GRAY)
    ret, x_t = cv2.threshold(x_t,1,255,cv2.THRESH_BINARY)
    x_t = x_t.copy().astype(np.float32) # image value must be float cause net needs the values to calculate gradients
    x_t = torch.from_numpy(x_t).unsqueeze(0)  # add one dimension representing the color channel
    return x_t.unsqueeze(0)  # add one dimension to form a batch

def plot_results(episode, episode_scores, losses, num_episodes, timestep):
    display.clear_output(wait = True)
    episode += 1
    f, ax = plt.subplots(1, 2, figsize=(12, 10))
    plt.subplot(1, 2, 1)
    plt.title("Scores: %s, episode %s" % (episode_scores[episode-1], episode))
    plt.xlabel("Episode")
    plt.xlim((0, num_episodes))
    plt.ylim(bottom=-0.2)
    plt.plot(episode_scores)
    
    plt.subplot(1, 2, 2)
    plt.title("Losses: timestep %s" % timestep)
    plt.xlabel("Step")
    plt.plot(losses)
    plt.show()
    

def plot_durations(episode_durations, average_num=100):
    plt.figure(2)
    plt.clf()
    durations_t = torch.tensor(episode_durations, dtype=torch.float)
    plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel('Duration')
    plt.plot(durations_t.numpy())
    # Take 100 episode averages and plot them too
    if len(durations_t) >= average_num:
        means = durations_t.unfold(0, average_num, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(average_num-1), means))
        plt.plot(means.numpy())
        
    plt.show()
#     plt.pause(0.001)  # pause a bit so that plots are updated
#     if is_ipython:
#     display.clear_output(wait=True)
#     display.display(plt.gcf())


In [ ]:
"""BATCH_SIZE = 128
GAMMA = 0.999
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 0.001
TARGET_UPDATE = 10
lr = 0.001
memory_size = 100000
num_episodes = 1000"""


def train(env, num_actions, in_channels, memory_size=100000, screen_shape=(84, 84), target_update=10, 
          BATCH_SIZE=128, GAMMA=0.999, EPS_START=0.9, EPS_END=0.05, EPS_DECAY=0.001, lr=0.001, num_episodes=1000,
         save_point=[500, 1000, 3000, 5000, 10000, 50000, 100000]):
    env = env
    saving_path = './model_saving'
    save_point = save_point
    
    # if GPU is available, use it otherwise use CPU
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    policy_net = DQN(num_actions, in_channels, screen_shape).to(device)
    target_net = DQN(num_actions, in_channels, screen_shape).to(device)
    
    # set weight and bias of target net as policy net
    target_net.load_state_dict(policy_net.state_dict())
    target_net.eval()
    
    optimizer = optim.Adam(params=policy_net.parameters(), lr=lr)
    
    memory = ReplayBuffer(memory_size, screen_shape)
    
    episode_durations = []
    scores = []
    losses = []
    last_screen = []
    timestep = 0
    
    for episode in range(num_episodes):
#         start_act = np.zeros(num_actions)
#         x_t, r, _ = env.frame_step(start_act)
#         x_t = preprocess(x_t, screen_shape)
#         print(x_t.shape)
#         x_t0, r0, terminal = env.frame_step(start_act)
#         x_t0 = preprocess(x_t0, screen_shape)
#         state = x_t0 - x_t
#         last_screen.append(x_t)
        
        
#         x_t = cv2.cvtColor(cv2.resize(x_t, (84, 84)), cv2.COLOR_BGR2GRAY)  # resize the screen and convert color to gray
#         ret, x_t = cv2.threshold(x_t,1,255,cv2.THRESH_BINARY)  # set the background to black and tetriminos to white
# #         s_t = np.stack((x_t, x_t, x_t, x_t), axis = 2)
#         print("episode", episode)
        survive_points = 0
        score = 0
        x_t0 = env.reset()
#         print("x_t0 size", x_t0.shape)
        
        while True:
            timestep += 1
            
#             x_t = cv2.cvtColor(cv2.resize(x_t, (84, 84)), cv2.COLOR_BGR2GRAY)
#             ret, x_t = cv2.threshold(x_t,1,255,cv2.THRESH_BINARY)
# #             x_t = np.array([0.0 if (x//255) < 1 else 255.0 for x in x_t])
            
# #             x_t = np.array(x_t, dtype=np.double)  # image value must be float cause net needs the values to calculate gradients
#             x_t = x_t.copy().astype(np.float32)
#             x_t = torch.from_numpy(x_t).unsqueeze(0)
#             x_t0, r_0, terminal = env.frame_step(start_act)
#             x_t0 = preprocess(x_t0, (84, 84))
#             x_t = last_screen[0]
            
#             print("time ", timestep)

            act = get_action(x_t0, policy_net)
#             print("int act", int(act))
            x_t1, r_1, terminal = env.step(int(act))
#             print("terminal", type(terminal))
            
        
#             cv2.imwrite("frame"+str(timestep)+".png", x_t1)
            
#             x_t1 = preprocess(x_t1, screen_shape)
            
            # Add extra reward if the agent survive
            score += r_1
            if not terminal:
                r_1 += 0.1
            
            
#             next_state = x_t1 - x_t0
            
            memory.push(x_t0, act, r_1, x_t1, terminal)
            
            print("score", score)
            
            x_t0 = x_t1
            
            
            if memory.can_sample(BATCH_SIZE):
                obs_batch, act_batch, rew_batch, next_obs_batch, done_mask = memory.sample(BATCH_SIZE)
#                 print("obs ", obs_batch.size())
#                 print("act ", type(act_batch))
#                 print("rew ", rew_batch.size())
                
                curr_qs = policy_net(obs_batch.to(device))
#                 print("curr size ", curr_qs.size())
                curr_qs = policy_net(obs_batch.to(device)).gather(1, act_batch.to(device))  # retrieve the max Q-value according to the index specified in act-batch
                next_qs = get_next_qs(target_net, next_obs_batch, done_mask, BATCH_SIZE)
#                 print("next qs ", next_qs.size())
#                 print("current qs ", curr_qs.size())
#                 print(curr_qs[0])
                
                target_q_values = rew_batch + GAMMA * next_qs
#                 print("target q ", target_q_values.size())
                
                criterion = nn.MSELoss()
                loss = criterion(curr_qs, target_q_values.unsqueeze(1))
                losses.append(loss)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                
                # Update target net at even interval 
                if timestep % target_update == 0:
                    target_net.load_state_dict(policy_net.state_dict())
                
            if terminal:
                episode_durations.append(timestep)
                scores.append(score)
                print(scores)
                plot_results(episode, scores, losses, num_episodes)
                plot_durations(episode_durations)
                break
                
            
                
            if episode in save_point:
                torch.save(policy_net, "%s/%s_%s.pth" % (saving_path, "DQN", episode))
                torch.save({
                            'episode': episode,
                            'model_state_dict': policy_net.state_dict(),
                            'target_state_dict': target_net.state_dict(),
                            'optimizer_state_dict': optimizer.state_dict(),
                            'loss': loss,
                            }, "%s/%s_%s_train.pth" % (saving_path, "DQN", episode))   # save for later training
                
        
                
            
                
            
                
            
            
            
        
                
            
            
            
        

In [ ]:
num_actions = 6
in_channels = 4  # due to frame stack
screen_shape = (84, 84)

BATCH_SIZE = 128
GAMMA = 0.999
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 0.001
TARGET_UPDATE = 100
FRAMESKIP = 4
lr = 0.001
memory_size = 100000
num_episodes = 50000

env = TetrisEnv()
env = TetrisPreprocessing(env, screen_size=84)
env = FrameStack(env,4)
# env.reset()
# o, r, d = env.step(1)
# print(o, r, d)
# y, r, t = env.frame_step(np.array([0, 0, 1, 0, 0, 0]))
# print(y)
# x = env.get_pixelscreen()
# env.save_screen()
train(env, num_actions, in_channels, memory_size, screen_shape, target_update = TARGET_UPDATE, num_episodes=num_episodes, save_point=[200, 300, 400, 500, 600, 700, 800, 900, 1000])

In [ ]:
import numpy as np
x = np.zeros([6])
# assert x.size == 7, (x.size, 7) 
class testing:
    def __init__(self):
        self.test = 123
        self.key = 'testing'
    def __array__(self):
        arr = self[:]
        print(arr)
    def __getitem__(self, item):
         return self.key
y = testing()
print(dir(y))
y.__array__()
z = y[:]
print(type(z))

a = np.array([1, 2, 3])
b = np.array([4, 5, 6])
np.array(np.stack((a, b)))

x = np.array([[1,2],[3,4]])
np.repeat(x, [2, 2], axis=0)
l = [9.8] * 3
x[np.newaxis, ...]